In [ ]:
# ===============================================================
# SEMI-DISCRETIZED
# ===============================================================
if discretization == "semi_discretized":
    R = - Bm + visc*Cm
    # Compute directional derivative about u in the direction of du (Jacobian)
    dF = derivative (R, variable, dvariable)
    dummy = (inner (Constant (0.0), v1) + inner (Constant (0.0), v2) + inner (Constant (0.0), v3) + inner (Constant (0.0), v4))*dx

####################
    # Assemble stiffness form
    A_stiffness = PETScMatrix ()
    b_stiffness = PETScVector ()

    if any ([system == "system_1", system == "system_2"]):    
        # assemble (dF, tensor = A_stiffness) # Alternative 1
        bcs = [] # Alternative 2
        A_stiffness, _ = assemble_system(dF, dummy, bcs = bcs, A_tensor = A_stiffness, b_tensor = b_stiffness) # Alternative 2

        A1 = matrix(A_stiffness.array ())
        condnumber = LA.cond (A1)
        print ("Condition number:", condnumber)

        # https://fenicsproject.org/docs/dolfin/1.4.0/python/programmers-reference/cpp/la/SLEPcEigenSolver.html
        # PETScOptions.set ("st_ksp_type", "preonly")
        # PETScOptions.set ("st_pc_type", "lu")
        # PETScOptions.set ("st_pc_factor_mat_solver_package", "mumps")

        # Create eigensolver
        eigensolver = SLEPcEigenSolver (A_stiffness) #(A, B)
        prm1 = eigensolver.parameters 
        PETScOptions.set ("eps_view")
        prm1["solver"]             = "subspace" # OPTIONS: "krylov-schur", "arnoldi", "power", "subspace"
        prm1["problem_type"]       = "non_hermitian" # OPTIONS: "gen_hermitian", "non_hermitian"
        # prm1["spectrum"]           = "target magnitude" # OPTIONS: "target magnitude", "largest magnitude"
        # prm1["spectral_transform"] = "shift-and-invert" # OPTIONS: "shift-and-invert"
        # prm1["spectral_shift"]     = 0.9
        # neigs = 12
    elif system == "system_3":   
        # assemble (dF, tensor = A_stiffness) # Alternative 1
        bcs = [] # Alternative 2
        A_stiffness, _ = assemble_system(dF, dummy, bcs = bcs, A_tensor = A_stiffness, b_tensor = b_stiffness) # Alternative 2

        A1 = matrix(A_stiffness.array ())
        condnumber = LA.cond (A1)
        print ("Condition number:", condnumber)

        # https://fenicsproject.org/docs/dolfin/1.4.0/python/programmers-reference/cpp/la/SLEPcEigenSolver.html
        # PETScOptions.set ("st_ksp_type", "preonly")
        # PETScOptions.set ("st_pc_type", "lu")
        # PETScOptions.set ("st_pc_factor_mat_solver_package", "mumps")

        # Create eigensolver
        eigensolver = SLEPcEigenSolver (A_stiffness) #(A, B)
        prm1 = eigensolver.parameters 
        PETScOptions.set ("eps_view")
        # prm1["solver"]             = "subspace" # OPTIONS: "krylov-schur", "arnoldi", "power", "subspace"
        prm1["problem_type"]       = "non_hermitian" # OPTIONS: "gen_hermitian", "non_hermitian"
        # prm1["spectrum"]           = "target magnitude" # OPTIONS: "target magnitude", "largest magnitude"
        # prm1["spectral_transform"] = "shift-and-invert" # OPTIONS: "shift-and-invert"
        # prm1["spectral_shift"]     = 0.9
        # neigs = 12

    # Compute all eigenvalues of A x = \lambda x
    eigensolver.solve () #(neigs)

    #Exporting the real part of the eigenvectors and plotting
    #eigenvalues
    Real = []
    Imag = []

    for i in range (eigensolver.get_number_converged ()):
        r, c, rv, cv = eigensolver.get_eigenpair (i)

        Real.append (r)
        Imag.append (c)

    # print (Real)
    plt.figure (5, figsize = (8, 4))
    fig, ax = plt.subplots ()
    area = area_stiffness
    ax.scatter (Real, Imag, s = area, marker = listmarkers [0], color = listcolor [4], edgecolors = listcolor [0], linewidths = 1.5, alpha = alphascatter)
    # ax.set_yscale ('symlog')

    plt.rcParams ['figure.figsize'] = mapsize
    # leg1 = ax.legend (loc = 'upper right', frameon = True, fontsize = 14);
    plt.grid (True, which = "both")
    plt.xlabel ('Re $[\mu]$ [1/s]', fontsize = 18)
    plt.ylabel ('Im $[\mu]$ [1/s]', fontsize = 16)
    # plt.xlim (-0.0025, -0.0015)
    # plt.ylim (-3, 3)
    matplotlib.rc ('xtick', labelsize = 18)     
    matplotlib.rc ('ytick', labelsize = 18)
    
    plt.axvline (0, label = 'pyplot vertical line', color = 'k')
    plt.show
    print ("Max Real", max (Real))
    print ("Min Real", min (Real))
    print ("Max Imag", max (Imag))
    print ("Min Imag", min (Imag))

    # Transform to numpy array
    A_mat = as_backend_type (A_stiffness).mat ()
    A_sparray = csr_matrix (A_mat.getValuesCSR ()[::-1], shape = A_mat.size)

    # Plot stiffness matrix
    plt.figure (6, figsize = (8, 4))
    fig, ax = plt.subplots ()
    plt.spy (A_sparray, color = 'k')
    plt.grid (True, which = "both")
    # # plt.xlim (0, l)
    # # ax.set_xlabel (r'L [m]', fontsize = 18)
    # # ax.set_ylabel (r'$p_i$ [Pa]', fontsize = 18)
    ax.xaxis.set_tick_params (which = 'major', size = 10, direction = 'in', top = 'on')
    ax.xaxis.set_tick_params (which = 'minor', size = 7, direction = 'in', top = 'on')
    ax.yaxis.set_tick_params (which = 'major', size = 10, direction = 'in', right = 'on')
    ax.yaxis.set_tick_params (which = 'minor', size = 7, direction = 'in', right = 'on')

    # Print mtrix
    print ("A_sparray = ", A_sparray)
